In [1]:
# import libraries
import requests
import pandas as pd
import time
import json
import pprint
from config import host_name, dbname, port, username, password, API_KEY
pp = pprint.PrettyPrinter(indent=2)

In [2]:
# youtube channel ID
CHANNEL_ID = "UCB2wtYpfbCpYDc5TeTwuqFA"

In [3]:
# API request 
pageToken = ""
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
response = requests.get(url).json()

In [4]:
pp.pprint(response)

{ 'etag': 'fLDvadktf3iA_P890PDpYMo57H8',
  'items': [ { 'etag': 'aE0uzF6wkP-QZGIcB6V4dM5A8Cg',
               'id': {'kind': 'youtube#video', 'videoId': '7T_7e4KKBSw'},
               'kind': 'youtube#searchResult',
               'snippet': { 'channelId': 'UCB2wtYpfbCpYDc5TeTwuqFA',
                            'channelTitle': 'Will Tennyson',
                            'description': 'Do you want to be a fitness '
                                           "influencer? In today's vlog I "
                                           'outline a typical day in the life '
                                           'of your average influencer! Hope '
                                           'you enjoy ...',
                            'liveBroadcastContent': 'none',
                            'publishTime': '2022-02-24T15:00:07Z',
                            'publishedAt': '2022-02-24T15:00:07Z',
                            'thumbnails': { 'default': { 'height': 90,
                    

In [5]:
video_id = response['items'][0]['id']['videoId']
video_id

'7T_7e4KKBSw'

In [6]:
video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')
video_title = str(video_title).replace('&quot;', '')
video_title = str(video_title).replace('&#39;', "'")
video_title

'How The Fitness Industry Is Scamming YOU!'

In [7]:
# Grab the upload date
upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0] # remove the timestamp and timezone
upload_date

'2022-02-24'

In [8]:
def get_video_details(video_id):
       # collecting view, like and comment counts with a second API call
        url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        
        return view_count, like_count, comment_count

In [9]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                video_title = str(video_title).replace("&#39;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [10]:
# create a dataframe
df2 = pd.DataFrame(columns= ['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])
df2 = get_videos(df2)
df2.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,7T_7e4KKBSw,How The Fitness Industry Is Scamming YOU!,2022-02-24,134741,10016,779
1,8ea_8bGLjJc,I Tried The Worlds Strictest Diet (According T...,2022-02-20,275959,14089,1241
2,3ayRRjn-NMc,Confessions Of A Fake Natty,2022-02-17,308621,18798,1227
3,iwh1CG49TXA,I Went On A 7 Day CRASH Diet! (10lbs in 1 Week!?),2022-02-10,414143,18516,1200
4,58al2-NpGO8,Living Like The Ultimate Gym &quot;BRO&quot;,2022-02-06,346821,18363,1151


In [11]:
# import the dataframe to an AWS database
!pip install psycopg2
import psycopg2 as ps


In [12]:
# Check for errors in connection to the database
def connect_to_db(host_name, dbname, port, username, password): 
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Beam me up, Scotty!')
    return conn

In [13]:
# conn to AWS db using imported credentials from config.py
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Beam me up, Scotty!


In [14]:
# create database table that looks like dataframe df2
df2

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,7T_7e4KKBSw,How The Fitness Industry Is Scamming YOU!,2022-02-24,134741,10016,779
1,8ea_8bGLjJc,I Tried The Worlds Strictest Diet (According T...,2022-02-20,275959,14089,1241
2,3ayRRjn-NMc,Confessions Of A Fake Natty,2022-02-17,308621,18798,1227
3,iwh1CG49TXA,I Went On A 7 Day CRASH Diet! (10lbs in 1 Week!?),2022-02-10,414143,18516,1200
4,58al2-NpGO8,Living Like The Ultimate Gym &quot;BRO&quot;,2022-02-06,346821,18363,1151
...,...,...,...,...,...,...
249,8XLIYeWIGRs,I Tried Jason Momoas Workout Diet For AQUAMAN...,2019-07-18,1583588,28240,1990
250,6H-h5nzT4bI,I Lived Like Mark Wahlberg For A Whole Day | 4...,2019-07-13,1994524,60179,3305
251,2_CRP6CnnIg,Letting Celebrities DECIDE What i Eat for 24 H...,2019-07-09,242862,9098,345
252,k2Vj40tJ4kE,Letting Homeless People DECIDE What i Eat For ...,2019-07-01,120657,9298,385


In [15]:
 # create database table, push command from notebook to my postgres DB on AWS
def create_table(curr):
    create_table_command = ('''CREATE TABLE IF NOT EXISTS videos (
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL
                    )''')
    curr.execute(create_table_command) #executes SQL command on python notebook

In [16]:
curr = conn.cursor() # allows python to execute SQL commands and fetch the results from the db

In [17]:
create_table(curr)

In [18]:
# executes SQL command onto the database
conn.commit()

In [19]:
# building the function for updating the database

# handling data for scalability

# check to see if video exists
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

# perform update on existing videos
# perform an insert on new videos
 

In [20]:
def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,)) # python needs video_id to be expressed as a tuple
    return curr.fetchone() is not None # fetchone() returns a single row from a table

In [21]:
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [22]:
new_vid_df = update_db(curr,df2)

In [23]:
# write SQL INSERT INTO command (just like the UPDATE command) to insert data into videos table
# wrap it into a function
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [24]:
# append data from df to SQL db

def append_from_df_to_db(curr, df2):
    for i, row in df2.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [25]:
new_vid_df = update_db(curr,df2)


In [26]:
tennydata = df2.to_csv('C:/Users/davez/Desktop/Youtube_Project/Youtube_Project/tenndydata.csv', index=False)

In [27]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [28]:
# data visualization

import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
import cufflinks as cf
import plotly.express as ps
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline

<function cufflinks.offline.go_offline(connected=None)>

In [37]:
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Bar(x=['video_title'], y=['like_count'])],
    layout=go.Layout(
        title=go.layout.Title(text="Views, Likes and Comments")
    )
)

fig.show()

In [32]:
df_views = df2.sort_values(by=['view_count'], ascending=True)
df_views

,video_id,video_title,upload_date,view_count,like_count,comment_count
148,WVI-jT-B7yw,I Tried The Keto Diet For A Week | Ketogenic D...,2020-09-24,1001415,33065,2672
124,g1ExW6d5PiY,Eating Training With Greg Doucette | Squat Ch...,2020-12-13,1013468,50475,3056
9,pIkbRaz_pSU,I Tried The Worlds Fittest Kids Diet Workout,2022-01-20,1071061,31112,2432
175,dzSBkK473mg,I Tried The Vertical Diet For A Week...,2020-06-21,1075208,34935,2191
198,CCl9b9eWkKs,Full Day of Eating Greg Doucette Style | ANABO...,2020-04-02,1078672,36606,1540
...,...,...,...,...,...,...
92,7VU2Dy96Btc,We Tried Chloe Tings Fat Loss Diet (Fake Eater?),2021-04-01,930577,39437,2787
186,8GFMs0AeeSg,I Tried The Fruitarian Diet | My Experience Go...,2020-05-14,947359,29440,2703
167,J7qPpo8edj8,Only Eating Anabolic ICE CREAM For 24 HOURS | ...,2020-07-19,949120,37732,2339
64,aMMulNeGzZ0,I Tried Michael Phelps Olympic Diet Training ...,2021-07-08,958475,31651,1923
